In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker
import requests
import json
from elasticsearch import Elasticsearch


In [2]:
class Story(object):
    pass
 
def loadSession():
    dbPath = 'stories.sqlite'
    engine = create_engine('sqlite:///%s' % dbPath, echo=True)
 
    bookmarks = Table('stories', MetaData(engine), autoload=True)
    mapper(Story, bookmarks)
 
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

session = loadSession()

2017-11-05 02:23:53,679 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-11-05 02:23:53,685 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 02:23:53,687 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-11-05 02:23:53,689 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 02:23:53,690 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stories")
2017-11-05 02:23:53,691 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 02:23:53,694 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'stories' AND type = 'table'
2017-11-05 02:23:53,695 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 02:23:53,696 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("stories")
2017-11-05 02:23:53,697 INFO sqlalchemy.engine.base.Engine ()
2017-11-05 02:23:53,698 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (

In [3]:
stories = session.query(Story).all()
print(len(stories))
print(dir(stories[0]))

2017-11-05 02:23:54,149 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-05 02:23:54,153 INFO sqlalchemy.engine.base.Engine SELECT stories.id AS stories_id, stories.title AS stories_title, stories.published AS stories_published, stories.tags AS stories_tags, stories.text AS stories_text, stories.likes AS stories_likes, stories.hrefs AS stories_hrefs, stories.url AS stories_url 
FROM stories
2017-11-05 02:23:54,155 INFO sqlalchemy.engine.base.Engine ()
23558
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_sa_class_manager', '_sa_instance_state', 'hrefs', 'id', 'likes', 'published', 'tags', 'text', 'title', 'url']


In [4]:
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "-L9u61j",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "7yPgKxlWRwmSpPqu9E9w2A",\n  "version" : {\n    "number" : "5.6.3",\n    "build_hash" : "1a2f265",\n    "build_date" : "2017-10-06T20:33:39.012Z",\n    "build_snapshot" : false,\n    "lucene_version" : "6.6.1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [6]:
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
   # es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    print(r.content)
    i=i+1
    break
 
print(i)

b'{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.co/api/planets/1/","films":["https://swapi.co/api/films/2/","https://swapi.co/api/films/6/","https://swapi.co/api/films/3/","https://swapi.co/api/films/1/","https://swapi.co/api/films/7/"],"species":["https://swapi.co/api/species/1/"],"vehicles":["https://swapi.co/api/vehicles/14/","https://swapi.co/api/vehicles/30/"],"starships":["https://swapi.co/api/starships/12/","https://swapi.co/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.co/api/people/1/"}'
2


In [10]:
es.get(index='zadolbali', id=5)

{'_id': '5',
 '_index': 'zadolbali',
 '_source': {'id': 5,
  'likes': 0,
  'published': 20090915,
  'tags': 'insurers mail',
  'text': 'Работаю в страховой компании. Любимые клиенты пишут:«На крышу мерседеса упала с балкона кошка. Cуществуют риски "падение предмета" и "атака животного". Что писать в страховом акте?»Претензионщики мрачно шутят, что если бы она упала и померла в полете — то было бы проще. Записали бы предметом.',
  'title': 'Атака одушевленного обьекта',
  'url': 'http://zadolba.li/story/5'},
 '_type': 'stories',
 '_version': 1,
 'found': True}

In [8]:
# 'hrefs', 'id', 'likes', 'published', 'tags', 'text', 'title', 'url'
for story in stories:
    body = {
        "id": story.id,
        "title": story.title,
        "text": story.text,
        "published": story.published,
        "likes": story.likes,
        "tags": story.tags,
        "url": story.url
    }
    es.index(index='zadolbali', doc_type='stories', id=story.id, body=body)

In [9]:
es.get(index='zadolbali', id=5)

{'_id': '5',
 '_index': 'zadolbali',
 '_source': {'id': 5,
  'likes': 0,
  'published': 20090915,
  'tags': 'insurers mail',
  'text': 'Работаю в страховой компании. Любимые клиенты пишут:«На крышу мерседеса упала с балкона кошка. Cуществуют риски "падение предмета" и "атака животного". Что писать в страховом акте?»Претензионщики мрачно шутят, что если бы она упала и померла в полете — то было бы проще. Записали бы предметом.',
  'title': 'Атака одушевленного обьекта',
  'url': 'http://zadolba.li/story/5'},
 '_type': 'stories',
 '_version': 1,
 'found': True}